# Validation

OpenFisca-UK runs unit and integration tests on each new version (see [here](https://github.com/PSLmodels/openfisca-uk/tree/master/tests)). In addition, the table below shows the aggregates produced by the model for the major taxes and benefits, and comparisons with UKMOD (latest [country report](https://www.iser.essex.ac.uk/research/publications/working-papers/cempa/cempa7-20.pdf)) and official sources[^1]. UKMOD and administrative sources refer to 2018, and OpenFisca-UK is simulated on policy at the end of 2018.

[^1]: From the UKMOD country report: unless otherwise specified: Department for Work and Pensions https://www.gov.uk/government/publications/benefit-expenditure-and-caseload-tables-2018 ; Best Start Grant: https://www2.gov.scot/Topics/Statistics/Browse/Social-Welfare/SocialSecurityforScotland/BSGJune2019; Child tax credit and working tax credit: HMRC statistics 
https://www.gov.uk/government/statistics/child-and-working-tax-credits-statistics-finalised-annual-awards-2016-to-2017; Scottish Child Payment: Scottish Fiscal Commission https://www.fiscalcommission.scot/forecast/supplementary-costing-scottish-child-payment; Scottish Child Winter Heating Assistance: Scottish Fiscal Commission 
https://www.fiscalcommission.scot/forecast/supplementary-costing-child-winter-heating-assistance; Income tax: HMRC statistics https://www.gov.uk/government/statistics/income-tax-liabilities-statistics-tax-year-2014-to-2015-to-tax-year-2017-to-2018; National Insurance Contributions: ONS Blue Book Table 5.2.4s 


In [2]:
from openfisca_uk import Microsimulation
from openfisca_uk_data import FRS_SPI_Adjusted
import pandas as pd

sim = Microsimulation()

VARIABLES = (
    "income_tax",
    "national_insurance",
    "employee_NI_class_1",
    "employer_NI_class_1",
    "NI_class_4",
    "universal_credit",
    "tax_credits",
    "housing_benefit",
    "state_pension",
)

df = pd.DataFrame(
    {
        "OpenFisca-UK": sim.df(VARIABLES, map_to="household", period=2019)
        .sum()
        .apply(lambda x: round(x / 1e9, 1))
    }
)
df["Variable"] = (
    "Income Tax",
    "Non-employer National Insurance",
    "Employee Class 1 NI",
    "Employer Class 1 NI",
    "Class 2 & 4 NI",
    "Universal Credit",
    "Tax Credits",
    "Housing Benefit",
    "State Pension",
)

_ = ""

df["UKMOD"] = (
    163.7,
    64.0,
    55.7,
    77.8,
    5.0,
    11.7,
    13.9,
    15.1,
    86.4,
)

df["External"] = (
    190.0,
    55.7,
    52.0,
    79.2,
    3.7,
    8.1,
    22.8,
    20.7,
    85.5,
)
df.set_index("Variable", drop=True, inplace=True)


def safe_ratio(x, y):
    try:
        return round(x / y, 2)
    except:
        return _


df["OpenFisca-UK / UKMOD"] = [
    safe_ratio(x, y) for x, y in zip(df["OpenFisca-UK"], df["UKMOD"])
]
df["OpenFisca-UK / External"] = [
    safe_ratio(x, y) for x, y in zip(df["OpenFisca-UK"], df["External"])
]

df
